In [1]:
! pip -q install transformers

In [2]:
from google.colab import drive
import os
import pandas as pd
import torch
from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


In [3]:
drive.mount('/content/drive/')
os.chdir("/content/drive/My Drive/Colab Notebooks")

Mounted at /content/drive/


In [4]:
data_val = pd.read_csv('reponse.csv')
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelWithLMHead.from_pretrained('TRAVIS')
model1 = AutoModelWithLMHead.from_pretrained('SAM')

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [5]:
ran=data_val.iloc[3000,4]
ran

" Oh, that's right, I suppose there is the matter of testimony."

In [6]:
import random

ran=data_val.iloc[random.randint(1, len(data_val)),random.randint(1,data_val.shape[1])]
print(ran)
dialog0=[]
dialog1=[]
dialog0.append(ran)
dialog1.append(ran)



for step in range(3):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(dialog0[step] + tokenizer.eos_token, return_tensors='pt')
    new_user_input_ids1 = tokenizer.encode(dialog1[step] + tokenizer.eos_token, return_tensors='pt')
    # append the new user input tokens to the chat history

    bot_input_ids0 = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    bot_input_ids1 = torch.cat([chat_history_ids1, new_user_input_ids1], dim=-1) if step > 0 else new_user_input_ids
    #print(bot_input_ids0)
    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids0, max_length=200,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=100, 
        top_p=0.7,
        temperature = 0.8
    )

    chat_history_ids1 = model1.generate(
        bot_input_ids1, max_length=200,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=100, 
        top_p=0.7,
        temperature = 0.8
    )
    wordsTravis=(tokenizer.decode(chat_history_ids[:, bot_input_ids0.shape[-1]:][0], skip_special_tokens=True))
    wordsSam=(tokenizer.decode(chat_history_ids1[:, bot_input_ids0.shape[-1]:][0], skip_special_tokens=True))
    dialog0.append(wordsTravis)
    dialog1.append(wordsSam)
    # pretty print last ouput tokens from bot
    print("TRAVIS: {}".format(wordsTravis))
    print("SAM: {}".format(wordsSam))

 Okay, yeah. Yeah, 13's okay. It wasn't terrible. I mean, it wasn't a terrible difficulty, so you just had to not roll low. 
TRAVIS:  Okay.  
SAM:  Okay, okay.  
TRAVIS:  All right, so that's your turn.  It's up to you.  Roll a d20. �ª 
SAM: .   It's a lot.  It's going to take some time, but I'm going to try to do it.  Let's see here.  Okay, you're up.  Yes, I'm up. I'm trying to get to the second level.  I'm not going to do that.  And I'm doing it. I think it's going well.  All right, so, as you all begin to step forward, the second one, you see this strange, pulsing light, and it's not quite as bright as you expected it to be.  You can see it begins to glow, and as it does, you can see the dark purple glow, as it begins the process of moving through the air


Input length of input_ids is 354, but ``max_length`` is set to 200. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


TRAVIS:  Oh, I have to roll a d10.  I have a d12.  And I will use my bonus action to-- 
SAM:  I'm not going to do that.  And I'm doing it. I think it's going well.  All right, so, as you all begin to step forward, the second one, you see this strange, pulsing light, and it's not quite as bright as you expected it to be.  You can see it begins to glow, and as it does, you can see the dark purple glow, as it begins the process of moving through the air.   It's a lot.  It's going to take some time, but I'm going to try to do it.  Let's see here.  Okay, you're up.  Yes, I'm up. I'm trying to get to the second level.  I'm not going to do that.  And I'm doing it. I think it's going well.  All right, so, as you all begin to step forward, the second one, you see this strange, pulsing light, and it's not quite as bright as you expected it to be.  You can see it begins to glow, and as it does, you can see the dark purple glow, as it begins the process of moving through the air Okay
